## Load useful libraries

In [ ]:
import pickle
import json

from tensorflow.keras.models import model_from_json

from plotting_utilities import plot_basic_loss

## Specify which training run we are investigating

In [ ]:
application_root_directory = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forecasting/deep_learning'
model_training_output_directory = application_root_directory + '/output/'

In [ ]:
#training_run = '57f47108-380d-4e78-aeb9-7ebccd07ec65----afc4d8d7-e500-4a07-9063-4e46cdecea7e'
#training_run = '57f47108-380d-4e78-aeb9-7ebccd07ec65----3946abec-9ef2-43b8-a0dd-930e0a93471e'

# 200 epochs, constant learning rate
#training_run = '57f47108-380d-4e78-aeb9-7ebccd07ec65----eb540bb2-a954-49c0-addc-bcbf85cda01b'

# 200 epochs, shrinking learning rate
training_run = '57f47108-380d-4e78-aeb9-7ebccd07ec65----4695ac3c-439c-4043-af60-b809d13dbca5'

In [ ]:
history_file = model_training_output_directory + '/' + training_run + '_final_history_regressor.pickled'

## Load the training history

In [ ]:
with open(history_file, 'rb') as f:
    history = pickle.load(f)

## Plot loss

In [ ]:
plot_basic_loss(history)

In [ ]:
model_json_filename = model_training_output_directory + '/' + training_run + '_model_regressor.json'
model_final_weights_filename = model_training_output_directory + '/' + training_run + '_final_weights_regressor.pickled'

# Load the JSON file containing the model architecture
with open(model_json_filename, 'r') as json_file:
    loaded_model_json = json_file.read()

# Create a new model from the JSON
model = model_from_json(loaded_model_json)

# Load the weights into the new model
model.load_weights(model_final_weights_filename)

In [ ]:
model

In [ ]:
filename_train_val_test_data = (
    model_training_output_directory 
    + training_run.split('----')[0]
    + '_train_val_test_dict.pickled'
)

with open(filename_train_val_test_data, 'rb') as f:
    train_val_test_dict = pickle.load(f)

In [ ]:
M_test = train_val_test_dict['test']['M']
y_test = train_val_test_dict['test']['y']
n_test = train_val_test_dict['test']['n']
position_test = train_val_test_dict['test']['position']

In [ ]:
M_test.shape

In [ ]:
y_predicted = model.predict(M_test)
y_predicted.shape

In [ ]:
import matplotlib.pyplot as plt

def plot_a_prediction(
    M,
    y,
    y_predicted,
    index = 5,
    n_y_forward = 15, # get this from somewhere else
    color_known_y = 'purple',
    color_predicted_y = 'magenta',
):

    X_all_scaled_index = 0  # this is the index for the main time series we are predicted

    x = M_test[index, :, X_all_scaled_index]
    y_known_min = y_test[index, 0]
    y_known_max = y_test[index, 1]
    y_predicted_min = y_predicted[index, 0]
    y_predicted_max = y_predicted[index, 1]

    n_full = len(x) + n_y_forward + 1
    xmin = 120. / n_full
    xmax = (120 + n_y_forward) / n_full

    plt.figure()
    plt.plot(x, label = 'Known Past')

    plt.axhline(y = y_known_min, xmin = xmin, xmax=xmax, color = color_known_y, label = 'Known Future')
    plt.axhline(y = y_known_max, xmin = xmin, xmax=xmax, color = color_known_y)

    plt.axhline(y = y_predicted_min, xmin = xmin, xmax=xmax, color = color_predicted_y, label = 'Predicted Future')
    plt.axhline(y = y_predicted_max, xmin = xmin, xmax=xmax, color = color_predicted_y)

    plt.xlim(0, n_full)
    
    plt.legend()
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
plot_a_prediction(M_test, y_test, y_predicted)

In [ ]:
plot_a_prediction(M_test, y_test, y_predicted, index = 1000)

In [ ]:
plot_a_prediction(M_test, y_test, y_predicted, index = 500)

In [ ]:
plot_a_prediction(M_test, y_test, y_predicted, index = 700)

In [ ]:
plot_a_prediction(M_test, y_test, y_predicted, index = 1200)